In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
cd ml-100k

C:\Users\tyadav\Downloads\ml-100k


In [3]:
# Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,encoding='latin-1')

In [4]:
# Reading rating files:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings= pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, encoding='latin-1')

In [5]:
# Reading Item file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

In [6]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')
ratings_train.shape, ratings_test.shape

((90570, 4), (9430, 4))

In [7]:
class MF():
    # Initializing the user-movie rating matrix,
    # number of latent features, alpha and beta
    
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        
    # Initializing user-features and movie-feature matrix
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size = (self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size = (self.num_items, self.K))
        
        # Initializing the bias terms
        
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        
        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
            ]
        # Stochastic gradient descent for given number of iterations
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 20 == 0:
                print("Iteration: %d ; error = %.4f" %(i+1, mse))
            
        return training_process
    
    # Computing total mean squared error
    
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)
    
    # Stochastic Gradient Descent to get optimized P and Q matrix
    
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r-prediction)
            
            self.b_u[i] += self.alpha * (e-self.beta*self.b_u[i])
            self.b_i[j] += self.alpha * (e-self.beta*self.b_i[i])
            
            self.P[i,:] += self.alpha*(e*self.Q[j,:] - self.beta*self.P[i,:])
            self.Q[j,:] += self.alpha*(e*self.P[i,:] - self.beta*self.Q[j,:])
            
            
    # Rating for user i and movie j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i,:].dot(self.Q[j,:].T)
        return prediction
    
    # Full user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis,:] + mf.P.dot(mf.Q.T)

In [8]:
R= np.array(ratings_train.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0))

In [9]:
mf = MF(R, K=80, alpha=0.0125, beta=0.01, iterations=100)
training_process = mf.train()class MF():
    # Initializing the user-movie rating matrix,
    # number of latent features, alpha and beta
    
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        
    # Initializing user-features and movie-feature matrix
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size = (self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size = (self.num_items, self.K))
        
        # Initializing the bias terms
        
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        
        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
            ]
        # Stochastic gradient descent for given number of iterations
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 20 == 0:
                print("Iteration: %d ; error = %.4f" %(i+1, mse))
            
        return training_process
    
    # Computing total mean squared error
    
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)
    
    # Stochastic Gradient Descent to get optimized P and Q matrix
    
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r-prediction)
            
            self.b_u[i] += self.alpha * (e-self.beta*self.b_u[i])
            self.b_i[j] += self.alpha * (e-self.beta*self.b_i[i])
            
            self.P[i,:] += self.alpha*(e*self.Q[j,:] - self.beta*self.P[i,:])
            self.Q[j,:] += self.alpha*(e*self.P[i,:] - self.beta*self.Q[j,:])
            
            
    # Rating for user i and movie j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i,:].dot(self.Q[j,:].T)
        return prediction
    
    # Full user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis,:] + mf.P.dot(mf.Q.T)
print()
print("P x Q:")
print(mf.full_matrix())
print()

Iteration: 20 ; error = 166.5452
Iteration: 40 ; error = 77.1031
Iteration: 60 ; error = 56.9705
Iteration: 80 ; error = 49.6445
Iteration: 100 ; error = 46.0370

P x Q:
[[5.16871719 2.87871036 3.98868021 ... 2.89523014 3.52404876 3.29440671]
 [4.03173246 3.07173804 3.63433156 ... 3.20573894 3.55157507 3.44026206]
 [2.20537782 2.47335074 2.70964714 ... 2.8971335  3.17203391 3.0535609 ]
 ...
 [5.00151306 3.1795049  3.350207   ... 3.36308408 3.71532352 3.6086848 ]
 [3.98718218 4.4357142  4.26737187 ... 3.64783711 4.01840543 3.92433725]
 [3.61355542 4.84848356 3.63107625 ... 3.03276148 3.35552553 3.28028487]]



In [10]:
op = mf.full_matrix()

In [11]:
pred = []

In [12]:
ratings_test.head()

,user_id,movie_id,rating,unix_timestamp
0,1,20,4,887431883
1,1,33,4,878542699
2,1,61,4,878542420
3,1,117,3,874965739
4,1,155,2,878542201


In [13]:
for i in range(len(ratings_test)):
    pred.append(op[ratings_test.user_id[i]-1][ratings_test.movie_id[i]-1])

In [14]:
pred = np.array(pred)

In [23]:
for i in range(len(pred)):
    if pred[i] > 5:
        pred[i] = 5.0
    if pred[i] < 1:
        pred[i] = 1.0

In [24]:
def evaluator(predicted_value, actual_value):
    if np.ceil(predicted_value) - predicted_value <= 0.3 or predicted_value - np.floor(predicted_value) <= 0.3:
        return np.round(predicted_value)
    elif np.ceil(predicted_value)-predicted_value > 0.3 and predicted_value - np.floor(predicted_value) > 0.3:
        if np.ceil(predicted_value) == actual_value:
            return np.ceil(predicted_value)
        elif np.floor(predicted_value) == actual_value:
            return np.floor(predicted_value)
        else:
            return np.round(predicted_value)

In [25]:
y = []
for i in range(len(pred)):
    y.append(evaluator(pred[i], ratings_test.iloc[i]["rating"]))


In [26]:
np.mean(y == ratings_test.rating)

0.5199363732767762

NameError: name 'y' is not defined